# Delta-X UAVSAR Workflow in NASA EarthData Cloud
Delta-X Uninhabited Aerial Vehicle Synthetic Aperture Radar (UAVSAR) datasets are available through NASA's Earthdata Cloud. NASA Earthdata on Cloud is always free and accessible via either HTTPS or direct [S3](https://aws.amazon.com/s3/) bucket access. With direct S3 access, you can bring your \"code to the data\", making your processing faster and scalable. Direct S3 access to NASA Earthdata on Cloud is only available if your Amazon Web Services ([AWS](https://aws.amazon.com/)) instance is set up in the `us-west-2` region. If you are new to the Earthdata Cloud, these NASA Earthdata [primers](https://www.earthdata.nasa.gov/learn/webinars-and-tutorials/cloud-primer-amazon-web-services) and [tutorials](https://www.earthdata.nasa.gov/learn/webinars-and-tutorials/how-cloud-earth-scientists) are good resources to get you started.

To access a Juypter Notebook on [AWS EC2](https://aws.amazon.com/ec2/) instance:,

1. On the AWS EC2 instance, start Juypter Notebook on port 8888 with the 'no-browser' parameter: `jupyter notebook --no-browser --port=8888`
1. On your local machine, forward port 8000 to the remote port 8888 : `ssh -i my-key-pair.pem -L 8000:localhost:8888 my-instance-user-name@my-instance-IPv6-address`
1. Now, the Jupyter Notebook can be accessed at `http://localhost:8000` in your local browser.

**Important**: [NASA Earthdata Login (EDL)](https://urs.earthdata.nasa.gov/) is required to obtain the S3 temporary credentials and direct access S3 objects bucket. First, set up NASA Earthdata Login authentication using a `.netrc` file. Please refer to the instructions here: https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget.

## Delta-X UAVSAR Datasets
ORNL DAAC has published following UAVSAR or UAVSAR-derived datasets collected for the Delta-X project.

**Delta-X:**
- UAVSAR L1 Single Look Complex (SLC) Stack Products, MRD, Louisiana, 2021. https://doi.org/10.3334/ORNLDAAC/1984
- UAVSAR L1B Interferometric Products, MRD, Louisiana, 2021. https://doi.org/10.3334/ORNLDAAC/1979
- UAVSAR L2 Interferometric Products, MRD, Louisiana, 2021. ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/2057
- UAVSAR Level 3 Geocoded InSAR Derived Water Level Changes, LA,USA, 2021. https://doi.org/10.3334/ORNLDAAC/2058

**Pre-Delta-X:**
- L1 UAVSAR Single Look Complex and Interferograms, MRD, LA, USA, 2016. https://doi.org/10.3334/ORNLDAAC/1816
- UAVSAR-derived Water Level Change Maps, Atchafalaya Basin, LA, USA, 2016. https://doi.org/10.3334/ORNLDAAC/1823
- UAVSAR Georeferenced Channel Maps, Atchafalaya Basin, LA, USA, 2016. https://doi.org/10.3334/ORNLDAAC/1954

In this tutorial we will read a Single Look Complex (SLC) image and visualize them.

In [ ]:
# import python modules
import requests
import s3fs
import numpy as np
import matplotlib.pyplot as plt

#### Data File Information
This dataset contains UAVSAR Level 1 (L1) Single Look Complex (SLC) stack products for Delta-X flight lines.
This L1 dataset is intended for users who are familiar with data from synthetic aperture radar, especially products from UAVSAR (https://uavsar.jpl.nasa.gov/). Contact UAVSAR for detailed information on how to interpret the files (https://uavsar.jpl.nasa.gov/cgi-bin/contact.pl).

These L1 data contain slant range single look complex (SLC), latitude/longitude/height, look vector, doppler, and metadata files. The data are provided in SLC stack format (*.slc) with associated annotation (*.ann), latitude-longitude-height (*.llh), look vector (*.lkv), and Doppler centroid-slant range (*.dop) files.  The single look complex (SLC) stacks are in the HH, HV, VH, and VV polarizations. The same area was sampled at approximately 30-minute intervals. The SLCs are not corrected for residual baseline (BU).

- Spatial Coverage:  Atchafalaya River and Terrebonne Basins in southern Louisiana
- Spatial Resolution: 0.8m (along- flight-line) by 1.7 m (slant range, along line-of-sight (LOS))
- Temporal Coverage:  2021-03-27 to 2021-04-18 and 2021-09-05 to 2021-09-13
- Temporal Resolution: estimates at 30-minute intervals
- File naming convention:
    - The SLC (*.slc) and annotation (*.ann) files are named according to the UAVSAR standard product file naming convention. Each SLC and annotation file name is in the following format:
    - {site name}_{line ID}_{flight ID}_{data take counter}_{acquisition date}_{band}{steering}{polarization}_{stack_number}_ {baseline correction}_{segment number}_{downsample factor}.slc
    - {site name}_{line ID}_{flight ID}_{data take counter}_{acquisition date}_{band}{steering}{polarization}_{stack_number}_ {baseline correction}.ann
    - The latitude-longitude-height (*.llh) files and look vector (*.lkv) files are named following the standard format:
    - {site name}_{line ID}_{stack number}_{baseline correction}_{segment number}_{downsample factor}.llh or .lkv
    - The Doppler centroid versus slant range (*.dop) files are named:
    - {site name}_{line ID}_{stack number}_{baseline correction}.dop
    - For all files, 
    - site name = "atchaf", "eterre" or "wterre"; 6-character alphanumeric site name assigned to the UAVSAR flight line.
    - line ID = 5-character flight line ID assigned to the UAVSAR flight line (Table 2, Fig 3). The first 3 characters are the aircraft heading in degrees from North, and the last 2 characters are an alphanumeric counter chosen to ensure uniqueness of the ID.
    - acquisition date = in format of YYMMDD, encoded as YY = the last two digits of the year, MM = month, DD = day of month, in UTC.

## Search for UAVSAR SLC granules and retrieve S3 links
Let's define a function that searches NASA's Common Metadata Repository ([CMR](https://cmr.earthdata.nasa.gov/search)) API to search and find the granules.

In [ ]:
def search_cmr(doi: str):
    # CMR API base url
    cmrurl='https://cmr.earthdata.nasa.gov/search/' 
    # doi search to get concept_id
    doisearch = f'{cmrurl}collections.json?doi={doi}'
    concept_id = requests.get(doisearch).json()['feed']['entry'][0]['id']

    page_num = 1
    page_size = 2000 # CMR page size limit
    s3_arr = []

    while True:
         # defining parameters
        cmr_param = {
            "collection_concept_id": concept_id, 
            "page_size": page_size,
            "page_num": page_num,
        }

        granulesearch = f'{cmrurl}granules.json'
        r = requests.post(granulesearch, data=cmr_param)
        r.raise_for_status()
        granules = r.json()['feed']['entry']
        if granules:
            for g in granules:
                # Get s3 links
                for links in g['links']:
                    if links['href'].startswith('s3://'):
                        s3_arr.append(links['href'])

            page_num += 1
        else: 
            break
            
    return s3_arr

The above python function search_cmr takes the dataset DOI and returns the dataset granules.

In [ ]:
doi = '10.3334/ORNLDAAC/1984'# DOI of UAVSAR L1 SLC prodcut

granule_arr = search_cmr(doi)

The granule_arr contains a list of s3 granule links, starting with `s3://`. We are interested in granule starting with `wterre_34202_21028_017_210407_L090VV_02_BU`. Let's print all these files.


In [ ]:
granule_sub = []
for g in granule_arr:
    if g.split('/')[-1].startswith("wterre_34202_21028_017_210407_L090VV_02_BU"):
        granule_sub.append(g)
print(granule_sub)

## Setup S3 Credentials and S3FS
A temporary S3 Credentials is needed for read-only, same-region, direct access to S3 objects on the Earthdata cloud. Each of the DAACs has its own endpoint. More information about S3 credentials for ORNL DAAC is available at: https://data.ornldaac.earthdata.nasa.gov/s3credentialsREADME.

We will now retrieve the credentials for ORNL DAAC data access. These credentials are temporary and currently valid for one hour.

In [ ]:
# python function to get s3 credentials
def get_s3credentials(daac: str):
    # DAAC S3 Credentials Endpoint
    earthadata_s3 = f"https://data.{daac}.earthdata.nasa.gov/s3credentials"
    r = requests.get(earthadata_s3)
    r.raise_for_status()
    return r.json()

# get s3 credentials
s3credentials = get_s3credentials("ornldaac")
# print expiration datetime
print(f'The S3 credentials will expire on {s3credentials["expiration"]} UTC')

We will use [S3Fs](https://s3fs.readthedocs.io/), Pythonic file interface to S3 for file-system style access. Alternatively, you can use [Boto3](https://aws.amazon.com/sdk-for-python/), AWS Software Development Kit (SDK) for Python. We will pass S3 credentials to S3Fs class S3FileSystem.

In [ ]:
fs_s3 = s3fs.S3FileSystem(anon=False, 
                          key=s3credentials['accessKeyId'], 
                          secret=s3credentials['secretAccessKey'], 
                          token=s3credentials['sessionToken'])

# Read and print the annotation file

In [ ]:
# open the image annotation file
ann_file = granule_sub[2]
with fs_s3.open(ann_file, mode='r') as fh:
    print(fh.read())

## Read SLC image

In [ ]:
# read SLC image. 

seg1 = granule_sub[1]
seg2 = granule_sub[0]

# Download files
# fs_s3.download(seg1, seg1.split('/')[-1])
# fs_s3.download(seg2, seg2.split('/')[-1])

# # Segment 1
# cols_s1 = 9900
# rows_s1 = 66664  
# ds = np.memmap(seg1.split('/')[-1], dtype=np.complex64)
# slc_seg1 = ds.reshape(rows_s1, cols_s1) ## multi-band example
# ds = None
# # Extract backscatter and phase
# backscatter_seg1 = np.abs(slc_seg1)
# phase_seg_1 = np.angle(slc_seg1)

# Segment 2
cols_s2 = 9900
rows_s2 = 55408  
ds = np.memmap(seg2.split('/')[-1], dtype=np.complex64)
slc_seg2 = ds.reshape(rows_s2, cols_s2) ## multi-band example
ds = None
# Extract backscatter and phase
backscatter_seg2 = np.abs(slc_seg2)
phase_seg_2 = np.angle(slc_seg2)

## Plot data

In [ ]:
# Plot data
fig = plt.figure(figsize=(18, 12))
# # display backscatter of the slc
# ax = fig.add_subplot(1,4,1)
# cax = ax.imshow(backscatter_seg1, vmin=0.05, vmax=0.35, cmap='gray')
# ax.set_title("backscatter segment1")
# cbar = fig.colorbar(cax, orientation='vertical')
# #display phase of the slc
# ax = fig.add_subplot(1,4,2)
# cax = ax.imshow(phase_seg_1, cmap='hsv')
# ax.set_title("phase segment1")
# cbar = fig.colorbar(cax, orientation='vertical')
# display backscatter of the slc
ax = fig.add_subplot(1,4,1)
cax = ax.imshow(backscatter_seg2, vmin=0.05, vmax=0.35, cmap='gray')
ax.set_title("backscatter segment2")
cbar = fig.colorbar(cax, orientation='vertical')
#display phase of the slc
ax = fig.add_subplot(1,4,2)
cax = ax.imshow(phase_seg_2, cmap='hsv')
ax.set_title("phase segment2")
cbar = fig.colorbar(cax, orientation='vertical')

plt.show()
plt.close("all")